# Getting Started with Symbolics.jl

*Heavily based on [this guide on the Symbolics.jl website](https://symbolics.juliasymbolics.org/stable/getting_started/).*

> Symbolics.jl is a symbolic modeling language. In this tutorial, we will walk you through the process of getting Symbolics.jl up and running, and start doing our first symbolic calculations.

## Installing Symbolics.jl

> Installing Symbolics.jl is as simple as using the Julia package manager. This is done via the command `]add Symbolics`. After precompilation is complete, do `using Symbolics` in the terminal (REPL) and when that command is completed, you're ready to start!

In [1]:
using Symbolics

## Building Symbolic Expressions

In [2]:
@variables x y  # Define x and y as symbolic variables via the @variables macro

2-element Vector{Num}:
 x
 y

In [3]:
z = x^2 + y # Define z as a symbolic expression (`istree` object)

y + x^2

In [4]:
A = [x^2 + y 0 2x
     0       0 2y
     y^2 + x 0 0]

3×3 Matrix{Num}:
 y + x^2  0  2x
       0  0  2y
 x + y^2  0   0

> Note that by default, @variables returns Sym or istree objects wrapped in Num to make them behave like subtypes of Real. Any operation on these Num objects will return a new Num object, wrapping the result of computing symbolically on the underlying values.

We can use normal Julia functions on Symbolics expressions. For example:

In [5]:
function f(u)
    [u[1] - u[3], u[1]^2 - u[2], u[3] + u[2]]
end

f([x, y, z])

3-element Vector{Num}:
 x - y - (x^2)
      -y + x^2
      2y + x^2

We can also use Julia functions to build symbolic expressions:

In [6]:
@variables u[1:3]
f(u)

3-element Vector{Num}:
    u[1] - u[3]
 -u[2] + u[1]^2
    u[2] + u[3]

## Derivatives

In [7]:
@variables t
D = Differential(t)
D.x     # The variable we are taking the derivative with respect to

t

The operator `D` that we just created is $D = \frac{\partial}{\partial t}$.

In [8]:
z = sin(t) + cos(t)
D(z)    # Derivative of z with respect to t

Differential(t)(cos(t) + sin(t))

The operator `D` is lazy, so we haven't computed anything. This is just a symbolic representation of $\frac{\partial}{\partial t}z$.  
In order to expand it, we may use:

In [9]:
expand_derivatives(D(z))

cos(t) - sin(t)

In [10]:
function wronskian(f, g, x)
    D = Differential(x)
    simplify(expand_derivatives(abs(D(f) * g - f * D(g))))
end

x = cos(t)
y = sin(t)

W = wronskian(x, y, t)

abs(-(cos(t)^2) - (sin(t)^2))

## Simplification and Substitution

> Symbolics interfaces with SymbolicUtils.jl to allow for simplifying symbolic expressions. Symbolics interfaces with SymbolicUtils.jl to allow for simplifying symbolic expressions.

In [11]:
@variables x y z t

display(simplify(2x + 2y))
display(simplify(cos(t)^2 + sin(t)^2))  # Trigonometric identities!
B = simplify.([
    t + t^2 + t + t^2
    2t + 2t^2
    x + y + y + 2t
    x + y - 2x + x + 0.5y - 1.5y
])

2(x + y)

1

4-element Vector{Num}:
   2(t + t^2)
   2(t + t^2)
 2(t + y) + x
            0

Then we can substitute values for the variables:

In [12]:
V = simplify.(substitute.(B, (Dict(x => 1, y => 2, t => 3),)))

4-element Vector{Num}:
 24
 24
 11
  0

In [13]:
Symbolics.value.(V)

4-element Vector{Int64}:
 24
 24
 11
  0

## `Sym`s and callable `Sym`s

In [14]:
@variables t x(t) y(t)

3-element Vector{Num}:
    t
 x(t)
 y(t)

Here, `t` is of type `Sym{Real}`.  
The name `x` refers to an object that represents the `Term` `x(t)`.

This expression defines `t` as an independent variable, and `x(t)` and `y(t)` as dependent variables.

In [15]:
z = x + y * t
expand_derivatives(D(z))

y(t) + Differential(t)(x(t)) + t*Differential(t)(y(t))

In [16]:
@variables g(..)

1-element Vector{Symbolics.CallWithMetadata{SymbolicUtils.FnType{Tuple, Real}, Base.ImmutableDict{DataType, Any}}}:
 g⋆

> Here, `g` is a variable that is a function of other variables. Any time that we reference `g` we have to utilize it as a function:

In [17]:
z = g(x) + g(y)

g(y(t)) + g(x(t))

## Registering Functions

Let us register a new symbolic function `h`.

In [18]:
h(x, y) = x^2 + y   # This is a standard Julia function
@register_symbolic h(x, y)

Observe that, since `h` is registered as a symbolic function, it doesn't expand to `x^2 + y`.

In [19]:
h(x, y) + y^2

h(x(t), y(t)) + y(t)^2

If we want to use it in the differentiation system, we need to register its derivatives:

In [20]:
# Derivative w.r.t. the first argument
Symbolics.derivative(::typeof(h), args::NTuple{2,Any}, ::Val{1}) = 2args[1]
# Derivative w.r.t. the second argument
Symbolics.derivative(::typeof(h), args::NTuple{2,Any}, ::Val{2}) = 1

display(h(x, y) + y^2)
display(Symbolics.derivative(h(x, y) + y^2, x))
display(Symbolics.derivative(h(x, y) + y^2, y))

h(x(t), y(t)) + y(t)^2

2x(t)

1 + 2y(t)

## Building Functions

We can build Julia functions from symbolic expressions using `build_function`.

In [21]:
to_compute = [x^2 + y, y^2 + x]
f_expr = build_function(to_compute, [x, y])
Base.remove_linenums!.(f_expr)

(:(function (ˍ₋arg1,)
      begin
          begin
              (SymbolicUtils.Code.create_array)(typeof(ˍ₋arg1), nothing, Val{1}(), Val{(2,)}(), (+)(ˍ₋arg1[2], (^)(ˍ₋arg1[1], 2)), (+)(ˍ₋arg1[1], (^)(ˍ₋arg1[2], 2)))
          end
      end
  end), :(function (ˍ₋out, ˍ₋arg1)
      begin
          begin
              #= /home/zen/.julia/packages/SymbolicUtils/r1pzW/src/code.jl:422 =# @inbounds begin
                      ˍ₋out[1] = (+)(ˍ₋arg1[2], (^)(ˍ₋arg1[1], 2))
                      ˍ₋out[2] = (+)(ˍ₋arg1[1], (^)(ˍ₋arg1[2], 2))
                      nothing
                  end
          end
      end
  end))

In [22]:
myf = eval(f_expr[1])
myf([2, 3])

2-element Vector{Int64}:
  7
 11

> The first is a function f([x, y]) that computes and builds an output vector [x^2 + y, y^2 + x]. Because this tool was made to be used by all the cool kids writing fast Julia codes, it is specialized to Julia and supports features like StaticArrays. For example:

In [23]:
using StaticArrays
myf(SA[2.0, 3.0])

2-element SVector{2, Float64} with indices SOneTo(2):
  7.0
 11.0

> The second function is an in-place non-allocating mutating function which mutates its first value. Thus, we'd use it like the following:

In [24]:
myf! = eval(f_expr[2])
out = zeros(2)
myf!(out, [2.0, 3.0])
out

2-element Vector{Float64}:
  7.0
 11.0

> To save the symbolic calculations for later, we can take this expression and save it out to a file:

In [25]:
write("function.jl", string(f_expr[2]))  # Writes the function to a file!

370

This essentially allows us to automatically write Julia code using symbolic expressions.